In [1]:
import warnings
warnings.filterwarnings('ignore')

import torch
from torch.utils.data import TensorDataset
from torch.utils.data import DataLoader
from torch.utils.data import Dataset

import torch.nn.functional as F
import torch.nn as nn
from torch.autograd import Variable

import torchvision
from torchvision import transforms
from torchsummary import summary

from albumentations import (
    ToFloat, 
    CLAHE, 
    RandomRotate90, 
    Transpose, 
    ShiftScaleRotate, 
    Blur, 
    OpticalDistortion, 
    GridDistortion, 
    HueSaturationValue, 
    IAAAdditiveGaussianNoise, 
    GaussNoise, 
    MotionBlur, 
    MedianBlur, 
    IAAPiecewiseAffine, 
    IAASharpen, 
    IAAEmboss, 
    RandomContrast, 
    RandomBrightness, 
    Flip, 
    OneOf, 
    Compose
)

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from pylab import rcParams
import numbers
from sklearn.model_selection import train_test_split

from PIL import Image
import math
from tqdm import tqdm

import os
from tqdm import tqdm_notebook as tqdm

# Ignore  the warnings
import warnings
warnings.filterwarnings('always')
warnings.filterwarnings('ignore')

import numpy as np
import pandas as pd

import warnings
import matplotlib.pyplot as plt
from matplotlib.pyplot import subplots
import matplotlib.patches as patches
import seaborn as sns
from pylab import rcParams

%matplotlib inline 
plt.style.use('seaborn')
sns.set(style='whitegrid',color_codes=True)

# classifiaction 
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier 
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
import xgboost as xgb
import catboost as ctb

# for classification
from sklearn.metrics import accuracy_score

# model selection
from sklearn.model_selection import train_test_split, cross_val_score, StratifiedKFold
from sklearn.model_selection import GridSearchCV

# Hp optimization imports
from hyperopt import STATUS_OK, Trials, fmin, hp, tpe
import mlflow

import re
import eli5
import gc
import random    
import math
import psutil
import pickle
import datetime
from time import time

# save/load models
from joblib import dump
from joblib import load


import numpy
from keras.datasets import imdb
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers.embeddings import Embedding
from keras.preprocessing import sequence
# fix random seed for reproducibility
numpy.random.seed(7)

/Users/kirillvladimirov/PycharmProjects/cv_projects/venv/lib/python3.7/site-packages/eli5/base_utils.py:28: DeprecationWarning: inspect.getargspec() is deprecated since Python 3.0, use inspect.signature() or inspect.getfullargspec()
  init_args = inspect.getargspec(class_.__init__)
/Users/kirillvladimirov/PycharmProjects/cv_projects/venv/lib/python3.7/site-packages/eli5/base_utils.py:36: DeprecationWarning: The usage of `cmp` is deprecated and will be removed on or after 2021-06-01.  Please use `eq` and `order` instead.
  return attr.s(class_, these=these, init=False, slots=True, **attrs_kwargs)  # type: ignore
Using TensorFlow backend.
/Users/kirillvladimirov/PycharmProjects/cv_projects/venv/lib/python3.7/site-packages/botocore/awsrequest.py:624: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated, and in 3.8 it will stop working
  class HeadersDict(collections.MutableMapping):


In [2]:
root = "../../data/raw/Gamma_Log_Facies_Type_Prediction/"
models_root = "../../models/Gamma_Log_Facies_Type_Prediction/"
RANDOM_STATE = 42
np.random.seed(RANDOM_STATE)
pd.set_option('max_columns', 150)
# rcParams['figure.figsize'] = 16,8

In [3]:
# Original code from https://www.kaggle.com/gemartin/load-data-reduce-memory-usage by @gemartin
# Modified to support timestamp type, categorical type
# Modified to add option to use float16 or not. feather format does not support float16.
from pandas.api.types import is_datetime64_any_dtype as is_datetime
from pandas.api.types import is_categorical_dtype

def reduce_mem_usage(df, use_float16=False):
    """ iterate through all the columns of a dataframe and modify the data type
        to reduce memory usage.        
    """
    start_mem = df.memory_usage().sum() / 1024**2
    print('Memory usage of dataframe is {:.2f} MB'.format(start_mem))
    
    for col in df.columns:
        if is_datetime(df[col]) or is_categorical_dtype(df[col]):
            # skip datetime type or categorical type
            continue
        col_type = df[col].dtype
        
        if col_type != object:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)  
            else:
                if use_float16 and c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)
        else:
            df[col] = df[col].astype('category')

    end_mem = df.memory_usage().sum() / 1024**2
    print('Memory usage after optimization is: {:.2f} MB'.format(end_mem))
    print('Decreased by {:.1f}%'.format(100 * (start_mem - end_mem) / start_mem))
    
    return df

In [4]:
%%time
full_train_df = pd.read_csv(root + "Train_File.csv")
full_test_df = pd.read_csv(root + "Test_File.csv")
submit_df = pd.read_csv(root + "Submission_File.csv")


CPU times: user 3.06 s, sys: 489 ms, total: 3.55 s
Wall time: 3.59 s


In [5]:
from sklearn.preprocessing import StandardScaler

In [6]:
from sklearn.model_selection import StratifiedKFold, KFold, RepeatedKFold, train_test_split, GroupKFold, GroupShuffleSplit

In [7]:
from sklearn.preprocessing import LabelEncoder

In [8]:
full_train_df.head()

,row_id,well_id,GR,label
0,0,0,143.510000,0
1,1,0,112.790928,0
2,2,0,123.531856,0
3,3,0,111.692784,0
4,4,0,123.613712,0


In [9]:
full_train_df.shape

(4400000, 4)

In [12]:
full_train_df["GR"].values.reshape(-1, 1)

array([[143.51      ],
       [112.79092812],
       [123.53185623],
       ...,
       [142.73450409],
       [140.03725205],
       [152.4       ]])

In [13]:
scaler = StandardScaler()

In [14]:
full_train_df["GR"] = scaler.fit_transform(full_train_df["GR"].values.reshape(-1, 1))

In [15]:
full_train_df["GR"]

0          0.999835
1          0.102729
2          0.416402
3          0.070659
4          0.418792
             ...   
4399995    1.533939
4399996    0.974187
4399997    0.977188
4399998    0.898418
4399999    1.259455
Name: GR, Length: 4400000, dtype: float64

In [16]:
full_test_df["GR"] = scaler.transform(full_test_df["GR"].values.reshape(-1, 1))

In [17]:
full_test_df["GR"]

0          0.136578
1          0.339437
2          0.177252
3          0.279943
4          0.539166
             ...   
2199995    1.054338
2199996    0.922949
2199997    0.057674
2199998    1.156047
2199999    0.318224
Name: GR, Length: 2200000, dtype: float64

In [41]:
train = full_train_df.drop(["label"], axis=1)
test = full_test_df
y = full_train_df["label"]

In [50]:
from time import time

In [124]:
def train(model, train_loader):
    batch_loss = 0.0
    batch_corrects = 0.0
    model.train()
    
    for inputs, labels in train_loader:
        optimizer.zero_grad()
        
        outputs = model(inputs)
#         outputs = model(inputs.float())
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        preds = torch.max(outputs, 1)[1]
        batch_loss += loss.item()
        batch_corrects += torch.sum(preds == labels.data)
    
    return batch_loss/len(train_loader), batch_corrects.float()/len(train_dataset)

In [125]:
def evaluate(model, valid_loader):
    loss = 0.0
    corrects = 0.0
    
    model.eval()
    with torch.no_grad():
        for inputs, labels in valid_loader:
            inputs, labels = Variable(inputs), Variable(labels)
            outputs = model(inputs)
            _, indices = torch.max(outputs, 1)
#             outputs = model(inputs.float())
            loss += F.cross_entropy(outputs, labels, reduction='mean').item()
            pred = outputs.data.max(1, keepdim=True)[1]
            corrects += pred.eq(labels.data.view_as(pred)).cpu().sum()
               
    return loss/len(valid_loader), corrects.float()/len(valid_dataset)

In [126]:
def experiment(model, epochs, criterion, optimizer, scheduler):
    epoch_loss_history = []
    epoch_corrects_history = []
    val_loss_history = []
    val_corrects_history = []

    for epoch in range(epochs):
        epoch_loss, epoch_corrects = train(model, train_loader)
        val_loss, val_corrects = evaluate(model, valid_loader)

        epoch_loss_history.append(epoch_loss)
        epoch_corrects_history.append(epoch_corrects)
        val_loss_history.append(val_loss)
        val_corrects_history.append(val_corrects)

        print('epoch:', (epoch+1))
        print('training loss: {:.4f}, training acc {:.4f} '.format(epoch_loss, epoch_corrects.item()))
        print('validation loss: {:.4f}, validation acc {:.4f} '.format(val_loss, val_corrects.item()))
        scheduler.step(val_corrects)
        
    return epoch_loss_history, epoch_corrects_history, val_loss_history, val_corrects_history

In [127]:
def show_histori():
    plt.plot(epoch_loss_history, label='training loss')
    plt.plot(val_loss_history, label='validation loss')
    plt.legend()
    plt.show()
    
    plt.plot(epoch_corrects_history, label='training accuracy')
    plt.plot(val_corrects_history, label='validation accuracy')
    plt.legend()
    plt.show()

In [66]:
full_train_df.shape

(4400000, 4)

In [95]:
train = np.array([x.values[:,2:3].T for group, x in full_train_df.groupby('well_id')], dtype='float32')
target = np.array([x.values[:,3:].T for group, x in full_train_df.groupby('well_id')], dtype='float32')
test = np.array([x.values[:,2:3].T for group, x in full_test_df.groupby('well_id')], dtype='float32')

In [97]:
train.shape, target.shape, test.shape

((4000, 1, 1100), (4000, 1, 1100), (2000, 1, 1100))

In [98]:
train[5]

array([[-0.25124577,  0.42343712,  0.8884384 , ...,  1.0192404 ,
         1.8522066 ,  1.458623  ]], dtype=float32)

In [102]:
X_train, X_valid, y_train, y_valid = train_test_split(train, target, test_size=0.2, random_state=RANDOM_STATE)

In [103]:
X_train.shape, y_train.shape, X_valid.shape, y_valid.shape

((3200, 1, 1100), (3200, 1, 1100), (800, 1, 1100), (800, 1, 1100))

In [104]:
# X_train, y_train, X_valid, y_valid = train_test_split(np.array(full_train_df["GR"]), np.array(full_train_df["label"]),test_size=0.2, random_state=RANDOM_STATE)
# train_df_ts.drop(["label"], axis=1), train_df_ts["label"], \
#             valid_df_ts.drop(["label"], axis=1), valid_df_ts["label"]

In [106]:
train_set = torch.utils.data.TensorDataset(torch.FloatTensor(X_train), torch.LongTensor(y_train))
val_set = torch.utils.data.TensorDataset(torch.FloatTensor(X_valid), torch.LongTensor(y_valid))

batch_size=64
train_loader = torch.utils.data.DataLoader(train_set, batch_size=batch_size)
valid_loader = torch.utils.data.DataLoader(val_set, batch_size=batch_size)

In [136]:
%%time

class LSTMClassifier(nn.Module):
    def __init__(self, input_dim, hidden_dim, target_size):
        super(LSTMClassifier, self).__init__()
        
        self.hidden_dim = hidden_dim

        self.lstm = nn.LSTM(input_dim, hidden_dim)
        self.hidden2tag = nn.Linear(hidden_dim, target_size)
        
    def forward(self, sentence):
#         lstm_out, _ = self.lstm(embeds.view(len(sentence), 1, -1))
        lstm_out, _ = self.lstm(sentence)
        tag_space = self.hidden2tag(lstm_out.view(len(sentence), -1))
        tag_scores = F.log_softmax(tag_space, dim=1)
        return tag_scores

torch.manual_seed(RANDOM_STATE)
model = LSTMClassifier(1100, 64, 5)
epochs = 50
criterion = nn.CrossEntropyLoss(reduction='sum')
# criterion = nn.NLLLoss()
optimizer = torch.optim.Adam(model.parameters(), lr = 0.01)
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(
    optimizer,
    factor=0.8,
    min_lr=1e-6,
    mode="max",
    verbose=True,
    patience=1,
    threshold=1e-3
)


epoch_loss_history = []
epoch_corrects_history = []
val_loss_history = []
val_corrects_history = []

for epoch in range(epochs):
    batch_loss = 0.0
    batch_corrects = 0.0
    model.train()
    
    for inputs, labels in train_loader:
        optimizer.zero_grad()
        
        outputs = model(inputs)
        _, indices = torch.max(outputs, 1)
        print(outputs)
        print(outputs.size())
        print(labels)
        print(labels.size())
        print(indices)
        print(indices.size())
        l = criterion(outputs, labels)
        l.backward()
        optimizer.step()

        preds = torch.max(outputs, 1)[1]
        batch_loss += l.item()
        batch_corrects += torch.sum(preds == labels.data)

    epoch_loss, epoch_corrects = batch_loss/len(train_loader), batch_corrects.float()/len(train_dataset)
    
    loss = 0.0
    corrects = 0.0
    
    model.eval()
    with torch.no_grad():
        for inputs, labels in valid_loader:
            inputs, labels = Variable(inputs), Variable(labels)
            outputs = model(inputs)
            _, indices = torch.max(outputs, 1)
            loss += F.cross_entropy(outputs, labels, reduction='mean').item()
            pred = outputs.data.max(1, keepdim=True)[1]
            corrects += pred.eq(labels.data.view_as(pred)).cpu().sum()

    val_loss, val_corrects = loss/len(valid_loader), corrects.float()/len(valid_dataset)

    epoch_loss_history.append(epoch_loss)
    epoch_corrects_history.append(epoch_corrects)
    val_loss_history.append(val_loss)
    val_corrects_history.append(val_corrects)

    print('epoch:', (epoch+1))
    print('training loss: {:.4f}, training acc {:.4f} '.format(epoch_loss, epoch_corrects.item()))
    print('validation loss: {:.4f}, validation acc {:.4f} '.format(val_loss, val_corrects.item()))
    scheduler.step(val_corrects)
        
show_histori()

tensor([[-1.6960, -1.4710, -1.5356, -1.7307, -1.6379],
        [-1.9016, -1.3947, -1.4104, -1.6813, -1.7570],
        [-1.7250, -1.4566, -1.5224, -1.6819, -1.6897],
        [-1.8924, -1.1788, -1.7316, -1.6009, -1.8145],
        [-1.8171, -1.2968, -1.6683, -1.5888, -1.7640],
        [-1.8076, -1.5724, -1.2833, -1.6313, -1.8604],
        [-2.1081, -1.3262, -1.3959, -1.5951, -1.8168],
        [-1.9318, -1.2868, -1.3251, -1.8064, -1.9041],
        [-1.8237, -1.6777, -1.2704, -1.6625, -1.7072],
        [-1.7972, -1.5877, -1.4583, -1.5481, -1.6898],
        [-1.5349, -1.5581, -1.6357, -1.6532, -1.6726],
        [-1.6570, -1.3723, -1.7124, -1.6153, -1.7344],
        [-1.7870, -1.3619, -1.6063, -1.7029, -1.6422],
        [-1.5173, -1.7419, -1.6482, -1.5937, -1.5609],
        [-1.6261, -1.5005, -1.7735, -1.4916, -1.6845],
        [-1.7013, -1.6961, -1.4599, -1.7773, -1.4576],
        [-1.5454, -1.7383, -1.6894, -1.5286, -1.5631],
        [-1.7036, -1.6840, -1.3538, -1.6948, -1.6583],
        [-

RuntimeError: multi-target not supported at ../aten/src/THNN/generic/ClassNLLCriterion.c:22

In [51]:
class LSTMClassifier(nn.Module):

    def __init__(self, in_dim, hidden_dim, num_layers, dropout, bidirectional, num_classes, batch_size):
        super(LSTMClassifier, self).__init__()
        self.in_dim = in_dim
        self.hidden_dim = hidden_dim
        self.batch_size = batch_size
        self.bidirectional = bidirectional
        self.num_dir = 2 if bidirectional else 1
        self.num_layers = num_layers
        self.dropout = dropout

        self.lstm = nn.LSTM(
            input_size=self.in_dim, 
            hidden_size=self.hidden_dim, 
            num_layers=self.num_layers, 
            dropout=self.dropout/2, 
            bidirectional=self.bidirectional,
            batch_first=True
        )
        self.gru = nn.GRU(
            self.hidden_dim * 2, 
            self.hidden_dim, 
            bidirectional=self.bidirectional, 
            batch_first=True
        )

        self.fc = nn.Sequential(
            nn.Linear(4096, int(hidden_dim)),
            nn.SELU(True),
            nn.Dropout(p=dropout),
            nn.Linear(int(hidden_dim), num_classes),
        )

    def forward(self, x):

        x = x.permute(2, 0, 1)
        lstm_out, _ = self.lstm(x)
        gru_out, _ = self.gru(lstm_out)
        avg_pool_l = torch.mean(lstm_out.permute(1, 0, 2), 1)
        max_pool_l, _ = torch.max(lstm_out.permute(1, 0, 2), 1)
        
        avg_pool_g = torch.mean(gru_out.permute(1, 0, 2), 1)
        max_pool_g, _ = torch.max(gru_out.permute(1, 0, 2), 1)

        x = torch.cat((avg_pool_g, max_pool_g, avg_pool_l, max_pool_l), 1)
        y = self.fc(x)
        
        return y

In [52]:
def plot_training_process(df: pd.DataFrame(), epoch_col: str, value_columns: list, y_axis_name: str, title: str):
    
    # code mostly based on: https://altair-viz.github.io/gallery/multiline_tooltip.html
    plot_df = df.melt(id_vars=epoch_col, value_vars=value_columns, var_name='group', value_name=y_axis_name)
    plot_df[y_axis_name] = plot_df[y_axis_name].round(4)
    nearest = alt.selection(type='single', nearest=True, on='mouseover', fields=[epoch_col], empty='none')
    line = alt.Chart().mark_line(interpolate='basis').encode(
        x=f'{epoch_col}:Q',
        y=f'{y_axis_name}:Q',
        color='group:N',
    ).properties(
        title=title
    )

    # Transparent selectors across the chart. This is what tells us
    # the x-value of the cursor
    selectors = alt.Chart().mark_point().encode(
        x=f'{epoch_col}:Q',
        opacity=alt.value(0),
    ).add_selection(
        nearest
    )

    # Draw points on the line, and highlight based on selection
    points = line.mark_point().encode(
        opacity=alt.condition(nearest, alt.value(1), alt.value(0))
    )

    # Draw text labels near the points, and highlight based on selection
    text = line.mark_text(align='left', dx=5, dy=-5).encode(
        text=alt.condition(nearest, f'{y_axis_name}:Q', alt.value(' '))
    )

    # Draw a rule at the location of the selection
    rules = alt.Chart().mark_rule(color='gray').encode(
        x=f'{epoch_col}:Q',
    ).transform_filter(
        nearest
    )

    # Put the five layers into a chart and bind the data
    return alt.layer(line, selectors, points, rules, text,
              data=plot_df, width=600, height=300).interactive()

In [53]:
def train_net(train_loader, val_loader, patience, model, criterion, optimizer, scheduler, verbose, plot_training):
    valid_loss_min = np.Inf
    patience = patience
    # current number of epochs, where validation loss didn't increase
    p = 0
    # whether training should be stopped
    stop = False

    epochs = 20000
    training_logs = []

    for e in range(1, epochs + 1):
        # print(time.ctime(), 'Epoch:', e)
        train_loss = []
        train_acc = []
        model.train()
        for batch_i, (data, target) in enumerate(train_loader):
            data, target = data.cuda(), target.cuda()

            optimizer.zero_grad()
            output = model(data)
            loss = criterion(output, target)
            train_loss.append(loss.item())

            a = target.data.cpu().numpy()
            b = output.detach().cpu().numpy().argmax(1)
            train_acc.append(accuracy_score(a, b))

            loss.backward()
            optimizer.step()

        val_loss = []
        val_acc = []
        for batch_i, (data, target) in enumerate(val_loader):
            data, target = data.cuda(), target.cuda()
            output = model(data)

            loss = criterion(output, target)
            val_loss.append(loss.item()) 
            a = target.data.cpu().numpy()
            b = output.detach().cpu().numpy().argmax(1)
            val_acc.append(accuracy_score(a, b))

        if e % 100 == 0 and verbose:
            print(f'Epoch {e}, train loss: {np.mean(train_loss):.4f}, valid loss: {np.mean(val_loss):.4f}, train acc: {np.mean(train_acc):.4f}, valid acc: {np.mean(val_acc):.4f}')

        training_logs.append([e, np.mean(train_loss), np.mean(val_loss), np.mean(train_acc), np.mean(val_acc)])

        scheduler.step(np.mean(val_loss))

        valid_loss = np.mean(val_loss)
        if valid_loss <= valid_loss_min:
            # print('Validation loss decreased ({:.6f} --> {:.6f}).  Saving model ...'.format(valid_loss_min, valid_loss))
            torch.save(model.state_dict(), 'model.pt')
            valid_loss_min = valid_loss
            p = 0

        # check if validation loss didn't improve
        if valid_loss > valid_loss_min:
            p += 1
            # print(f'{p} epochs of increasing val loss')
            if p > patience:
                print('Stopping training')
                stop = True
                break        

        if stop:
            break

    checkpoint = torch.load('model.pt')      
    model.load_state_dict(checkpoint)
    
    if plot_training:
        training_logs = pd.DataFrame(training_logs, columns=['Epoch', 'Train loss', 'Valid loss', 'Train accuracy', 'Validation accuracy'])
        loss_plot = plot_training_process(df=training_logs, epoch_col='Epoch', value_columns=['Train loss', 'Valid loss'], y_axis_name='loss', title='Loss progress')
        acc_plot = plot_training_process(df=training_logs, epoch_col='Epoch', value_columns=['Train accuracy', 'Validation accuracy'], y_axis_name='accuracy', title='Accuracy progress')
        render(loss_plot & acc_plot)
    
    return model

In [54]:
def initialize_model():
    torch.manual_seed(42)
    model = LSTMClassifier(10, 512, 3, 0.1, True, 9, 64)
    criterion = nn.CrossEntropyLoss()
    optimizer = torch.optim.SGD(model.parameters(), lr=.5)
    model.cuda()
    scheduler = lr_scheduler.ReduceLROnPlateau(optimizer, patience=8, factor=0.5, verbose=True)
    return model, criterion, optimizer, scheduler

In [59]:
def train_net_folds(X, X_test, y, folds, plot_training, batch_size, patience, verbose):

    oof = np.zeros((len(X), 9))
    prediction = np.zeros((len(X_test), 9))
    scores = []
    for fold_n, (train_index, valid_index) in enumerate(folds.split(X, y)):
        print('Fold', fold_n, 'started at', time())
        print(train_index, valid_index)
        X_train, X_valid = X[train_index], X[valid_index]
        y_train, y_valid = y[train_index], y[valid_index]
        
        train_set = torch.utils.data.TensorDataset(torch.FloatTensor(X_train), torch.LongTensor(y_train))
        val_set = torch.utils.data.TensorDataset(torch.FloatTensor(X_valid), torch.LongTensor(y_valid))

        train_loader = torch.utils.data.DataLoader(train_set,batch_size=batch_size, shuffle=True)
        val_loader = torch.utils.data.DataLoader(val_set,batch_size=batch_size)
        
        model, criterion, optimizer, scheduler = initialize_model()
        model = train_net(train_loader, val_loader, patience, model, criterion, optimizer, scheduler, verbose, plot_training)
        
        y_pred_valid = []
        for batch_i, (data, target) in enumerate(val_loader):
            data, target = data.cuda(), target.cuda()
            p = model(data)
            pred = p.cpu().detach().numpy()
            y_pred_valid.extend(pred)
            
        y_pred = []
        for i, data in enumerate(test):
            p = model(torch.FloatTensor(data).unsqueeze(0).cuda())
            y_pred.append(p.cpu().detach().numpy().flatten())
            
        oof[valid_index] = np.array(y_pred_valid)
        scores.append(accuracy_score(y_valid, np.array(y_pred_valid).argmax(1)))

        prediction += y_pred

    prediction /= n_fold
    
    prediction = np.array(prediction).argmax(1)
    
    print('CV mean score: {0:.4f}, std: {1:.4f}.'.format(np.mean(scores), np.std(scores)))
    print('--' * 50)
    
    return oof, prediction

In [60]:
n_fold = 5
folds = StratifiedKFold(n_splits=n_fold, shuffle=True, random_state=42)
oof, prediction = train_net_folds(train, test, y, folds, True, 64, 40, True)

Fold 0 started at 1575616543.407255
[      0       2       3 ... 4399997 4399998 4399999] [      1      11      12 ... 4399971 4399973 4399978]


KeyError: "None of [Int64Index([      0,       2,       3,       4,       5,       6,       7,\n                  8,       9,      10,\n            ...\n            4399990, 4399991, 4399992, 4399993, 4399994, 4399995, 4399996,\n            4399997, 4399998, 4399999],\n           dtype='int64', length=3519998)] are in the [columns]"

In [153]:
lstm = nn.LSTM(input_size=3, hidden_size=3)  # Input dim is 3, output dim is 3
inputs = [torch.randn(1, 3) for _ in range(10)]  # make a sequence of length 5

# initialize the hidden state.
hidden = (torch.randn(1, 1, 3),
          torch.randn(1, 1, 3))
for i in inputs:
    # Step through the sequence one element at a time.
    # after each step, hidden contains the hidden state.
    out, hidden = lstm(i.view(1, 1, -1), hidden)

# alternatively, we can do the entire sequence all at once.
# the first value returned by LSTM is all of the hidden states throughout
# the sequence. the second is just the most recent hidden state
# (compare the last slice of "out" with "hidden" below, they are the same)
# The reason for this is that:
# "out" will give you access to all hidden states in the sequence
# "hidden" will allow you to continue the sequence and backpropagate,
# by passing it as an argument  to the lstm at a later time
# Add the extra 2nd dimension
inputs = torch.cat(inputs).view(len(inputs), 1, -1)
hidden = (torch.randn(1, 1, 3), torch.randn(1, 1, 3))  # clean out hidden state
out, hidden = lstm(inputs, hidden)

In [147]:
lstm

LSTM(3, 3)

In [155]:
inputs

tensor([[[-0.4208, -0.7492, -0.3161]],

        [[-0.3051,  0.3074,  0.7358]],

        [[ 1.9541, -0.7806, -0.0920]],

        [[ 2.1447, -2.1230, -1.2172]],

        [[-1.8088,  0.3749, -0.5533]],

        [[ 0.3254, -1.6975, -0.7601]],

        [[ 0.2054,  0.9976,  0.8525]],

        [[-0.5000, -0.4228, -0.3035]],

        [[-0.9898, -0.4275,  0.0078]],

        [[ 0.5273, -1.5449, -1.6076]]])

In [149]:
hidden

(tensor([[[0.1727, 0.2146, 0.0673]]], grad_fn=<StackBackward>),
 tensor([[[0.5718, 0.5690, 0.0858]]], grad_fn=<StackBackward>))

In [151]:
for i in inputs:
    print(i)
    print(i.view(1, 1, -1))
    break

tensor([[ 0.7703, -0.0500, -0.4288]])
tensor([[[ 0.7703, -0.0500, -0.4288]]])


In [156]:
inputs = torch.cat(inputs).view(len(inputs), 1, -1)

TypeError: cat(): argument 'tensors' (position 1) must be tuple of Tensors, not Tensor

In [157]:
out, hidden = lstm(inputs, hidden)

In [158]:
out

tensor([[[ 0.3851,  0.2380,  0.0597]],

        [[ 0.3642,  0.2471,  0.0179]],

        [[ 0.2595,  0.1205, -0.0535]],

        [[ 0.0935,  0.0618, -0.0790]],

        [[ 0.2935,  0.3007,  0.1061]],

        [[ 0.3879,  0.1461,  0.1010]],

        [[ 0.3556,  0.2804,  0.0649]],

        [[ 0.3897,  0.2230,  0.0902]],

        [[ 0.3557,  0.2325,  0.0667]],

        [[ 0.4465,  0.1172,  0.1175]]], grad_fn=<StackBackward>)

In [160]:
inputs.size()

torch.Size([10, 1, 3])

In [159]:
out.size()

torch.Size([10, 1, 3])

In [165]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

torch.manual_seed(1)

In [166]:
def prepare_sequence(seq, to_ix):
    idxs = [to_ix[w] for w in seq]
    return torch.tensor(idxs, dtype=torch.long)

In [167]:
training_data = [
    ("The dog ate the apple".split(), ["DET", "NN", "V", "DET", "NN"]),
    ("Everybody read that book".split(), ["NN", "V", "DET", "NN"])
]

word_to_ix = {}
for sent, tags in training_data:
    for word in sent:
        if word not in word_to_ix:
            word_to_ix[word] = len(word_to_ix)
print(word_to_ix)

tag_to_ix = {"DET": 0, "NN": 1, "V": 2}

# These will usually be more like 32 or 64 dimensional.
# We will keep them small, so we can see how the weights change as we train.
EMBEDDING_DIM = 6
HIDDEN_DIM = 6

{'The': 0, 'dog': 1, 'ate': 2, 'the': 3, 'apple': 4, 'Everybody': 5, 'read': 6, 'that': 7, 'book': 8}


In [168]:
class LSTMTagger(nn.Module):

    def __init__(self, embedding_dim, hidden_dim, vocab_size, tagset_size):
        super(LSTMTagger, self).__init__()
        self.hidden_dim = hidden_dim

        self.word_embeddings = nn.Embedding(vocab_size, embedding_dim)

        # The LSTM takes word embeddings as inputs, and outputs hidden states
        # with dimensionality hidden_dim.
        self.lstm = nn.LSTM(embedding_dim, hidden_dim)

        # The linear layer that maps from hidden state space to tag space
        self.hidden2tag = nn.Linear(hidden_dim, tagset_size)

    def forward(self, sentence):
        embeds = self.word_embeddings(sentence)
        lstm_out, _ = self.lstm(embeds.view(len(sentence), 1, -1))
        tag_space = self.hidden2tag(lstm_out.view(len(sentence), -1))
        tag_scores = F.log_softmax(tag_space, dim=1)
        return tag_scores

In [176]:
model = LSTMTagger(EMBEDDING_DIM, HIDDEN_DIM, len(word_to_ix), len(tag_to_ix))
loss_function = nn.NLLLoss()
optimizer = optim.SGD(model.parameters(), lr=0.1)

# See what the scores are before training
# Note that element i,j of the output is the score for tag j for word i.
# Here we don't need to train, so the code is wrapped in torch.no_grad()
with torch.no_grad():
    inputs = prepare_sequence(training_data[0][0], word_to_ix)
    tag_scores = model(inputs)
    print(tag_scores)

for epoch in range(2000):  # again, normally you would NOT do 300 epochs, it is toy data
    for sentence, tags in training_data:
        # Step 1. Remember that Pytorch accumulates gradients.
        # We need to clear them out before each instance
        model.zero_grad()

        # Step 2. Get our inputs ready for the network, that is, turn them into
        # Tensors of word indices.
        sentence_in = prepare_sequence(sentence, word_to_ix)
        targets = prepare_sequence(tags, tag_to_ix)
        print(len(sentence), len(tags))
        print(sentence_in.size(), targets.size())
        # Step 3. Run our forward pass.
        tag_scores = model(sentence_in)
        print(tag_scores.size())
        # Step 4. Compute the loss, gradients, and update the parameters by
        #  calling optimizer.step()
        loss = loss_function(tag_scores, targets)
        loss.backward()
        optimizer.step()
        break
    break
# See what the scores are after training
with torch.no_grad():
    inputs = prepare_sequence(training_data[0][0], word_to_ix)
    tag_scores = model(inputs)

    # The sentence is "the dog ate the apple".  i,j corresponds to score for tag j
    # for word i. The predicted tag is the maximum scoring tag.
    # Here, we can see the predicted sequence below is 0 1 2 0 1
    # since 0 is index of the maximum value of row 1,
    # 1 is the index of maximum value of row 2, etc.
    # Which is DET NOUN VERB DET NOUN, the correct sequence!
    print(tag_scores)

tensor([[-1.0514, -1.5035, -0.8481],
        [-0.9243, -1.5360, -0.9469],
        [-0.9562, -1.5631, -0.9010],
        [-1.0245, -1.5138, -0.8652],
        [-1.0046, -1.5902, -0.8441]])
5 5
torch.Size([5]) torch.Size([5])
torch.Size([5, 3])
tensor([[-1.0424, -1.4777, -0.8693],
        [-0.9196, -1.5053, -0.9693],
        [-0.9502, -1.5339, -0.9222],
        [-1.0169, -1.4845, -0.8875],
        [-0.9972, -1.5536, -0.8685]])


In [178]:
input_dim = 5
hidden_dim = 10
n_layers = 1

lstm_layer = nn.LSTM(input_dim, hidden_dim, n_layers, batch_first=True)

In [179]:
batch_size = 1
seq_len = 1

inp = torch.randn(batch_size, seq_len, input_dim)
hidden_state = torch.randn(n_layers, batch_size, hidden_dim)
cell_state = torch.randn(n_layers, batch_size, hidden_dim)
hidden = (hidden_state, cell_state)

In [180]:
inp

tensor([[[ 0.1731, -1.1531,  0.7127, -0.3057,  1.6711]]])

In [181]:
hidden_state

tensor([[[-1.6544, -1.7879,  0.4142,  0.8780, -0.8525, -0.9350,  0.2770,
          -0.5112,  0.4880, -0.0164]]])

In [182]:
inp = torch.randn(batch_size, seq_len, input_dim)
out, hidden = lstm_layer(inp, hidden)

In [183]:
from keras.layers import Dense, Flatten, LSTM, Activation
from keras.layers import Dropout, RepeatVector, TimeDistributed
from keras import Input, Model

seq_length = 1100
input_dims = 64
output_dims = 5 # number of classes
n_hidden = 64
model1_inputs = Input(shape=(seq_length,input_dims,))
model1_outputs = Input(shape=(output_dims,))

net1 = LSTM(n_hidden, return_sequences=True)(model1_inputs)
net1 = LSTM(n_hidden, return_sequences=False)(net1)
net1 = Dense(output_dims, activation='relu')(net1)
model1_outputs = net1

model1 = Model(inputs=model1_inputs, outputs = model1_outputs, name='model1')

## Fit the model
model1.summary()

Model: "model1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 1100, 64)          0         
_________________________________________________________________
lstm_1 (LSTM)                (None, 1100, 64)          33024     
_________________________________________________________________
lstm_2 (LSTM)                (None, 64)                33024     
_________________________________________________________________
dense_1 (Dense)              (None, 5)                 325       
Total params: 66,373
Trainable params: 66,373
Non-trainable params: 0
_________________________________________________________________
